![1](images/DIPOLE_DESSERT_300_1.png)
    
![2](images/DIPOLE_DESSERT_300_2.png)

![3](images/DIPOLE_DESSERT_300_3.png)
<!--     
![4](images/DIPOLE_DESSERT_300_4.png) -->

In [2]:
import json
import pennylane as qml
import pennylane.numpy as np
# Write any helper functions you need here
def true_GHZ(n_qubits):
    qml.Hadamard(0)
    for i in range(n_qubits-1):
        qml.CNOT([i, i+1])
    
    return qml.state()
def GHZ_circuit(noise_param, n_qubits):

    """
    Quantum circuit that prepares an imperfect GHZ state using gates native to a neutral atom device.

    Args:
        - noise_param (float): Parameter that quantifies the noise in the CZ gate, modelled as a 
        depolarizing channel on the target qubit. noise_param is the parameter of the depolarizing channel
        following the PennyLane convention.
        - n_qubits (int): The number of qubits in the prepared GHZ state.
    Returns:
        - (np.tensor): A density matrix, as returned by `qml.state`, representing the imperfect GHZ state.
    
    """
    

    # Put your code here
    for i in range(n_qubits):
        qml.RY(np.pi/2, i)
        qml.RX(np.pi, i)
        
    for i in range(n_qubits-1):
        qml.CZ([i,i+1])
        qml.DepolarizingChannel(noise_param, i+1)
        qml.RY(np.pi/2, i+1)
        qml.RX(np.pi, i+1)
        
    return qml.state()    

def GHZ_fidelity(noise_param, n_qubits):

    """
    Calculates the fidelity between the imperfect GHZ state returned by GHZ_circuit and the ideal GHZ state.

    Args:
        - noise_param (float): Parameter that quantifies the noise in the CZ gate, modelled as a 
        depolarizing channel on the target qubit. noise_param is the parameter of the depolarizing channel
        following the PennyLane convention.
        - n_qubits (int): The number of qubits in the GHZ state.
    Returns:
        - (float): The fidelity between the noisy and ideal GHZ states.
    """
    
    dev = qml.device('default.mixed', wires=n_qubits)
    
    GHZ_QNode = qml.QNode(GHZ_circuit,dev)
    

    # Use GHZ_QNode to find the fidelity between
    # the noisy GHZ state and an ideal GHZ state
    true_GHZ_QNode = qml.QNode(true_GHZ,dev)
    return qml.math.fidelity(GHZ_QNode(noise_param, n_qubits), true_GHZ_QNode(n_qubits))   

# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    output = GHZ_fidelity(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    
    dev = qml.device('default.mixed', wires=4)
    qnode = qml.QNode(GHZ_circuit, dev)
    u = qnode(0.05,3)
    
    for op in qnode.tape.operations:
        assert (isinstance(op, qml.RX) or isinstance(op, qml.RY) or isinstance(op, qml.CZ) or isinstance(op, qml.DepolarizingChannel)), "You are using forbidden gates!"

    assert np.isclose(solution_output, expected_output, rtol = 1e-4)
